In [1]:
# aim: runn dnn hmm hybrid using previous code, then can condense into normal classifier structure

# get data:

import sys
sys.path.append("/Users/david/Documents/mastersCode/ubm")
print(sys.path)

['/Users/david/Documents/mastersCode/ubm/classifiers/hmm-hybrids/dnnhmm/development', '/Users/david/.conda/envs/ubm/lib/python310.zip', '/Users/david/.conda/envs/ubm/lib/python3.10', '/Users/david/.conda/envs/ubm/lib/python3.10/lib-dynload', '', '/Users/david/.conda/envs/ubm/lib/python3.10/site-packages', '/Users/david/Documents/mastersCode/ubm']


In [5]:
# test for gmm-ubm on whale calls
from importlib import reload
import audio_datastore.audio_datastore as myads
reload(myads)
from collections import Counter
import numpy as np
import config
import librosa
import utils as my_utils
import matplotlib.pyplot as plt
import feature_extraction.fe_spafe.fe_spafe as fe_spafe
import feature_extraction.fe_base as fe_base
import os.path
reload(fe_base)
reload(fe_spafe)

<module 'feature_extraction.fe_spafe.fe_spafe' from '/Users/david/Documents/mastersCode/ubm/feature_extraction/fe_spafe/fe_spafe.py'>

In [3]:
# create ads
# female ads
datasetFolder = r"/Users/david/Documents/data/speech/ivectors/SPEECH DATA/FEMALE/MIC"
female_ads = myads.AudioDatastore()
female_ads.populate(datasetFolder, include_sub_folders=True, label_source=True)
Counter(female_ads.labels).values()
# male ads
datasetFolder = r"/Users/david/Documents/data/speech/ivectors/SPEECH DATA/MALE/MIC"
male_ads = myads.AudioDatastore()
male_ads.populate(datasetFolder, include_sub_folders=True, label_source=True)
Counter(male_ads.labels).values()
# combine ads
ads = myads.AudioDatastore()
ads.set([datasetFolder], files=female_ads.files + male_ads.files, labels=female_ads.labels + male_ads.labels)
# split ads
developmentLabels = ["M01", "M02", "M03", "M04", "M06", "M07", "M08", "M09", "F01", "F02", "F03", "F04", "F06", "F07",
                     "F08", "F09"]
evaluationLabels = ["M05", "M10", "F05", "F10"]
adsTrain = myads.subset(ads, developmentLabels)
adsEvaluate = myads.subset(ads, evaluationLabels)
# numFilesPerSpeakerForEnrollment = 3
adsEnroll, adsTest = myads.split(adsEvaluate, 5)
adsTest, _ = myads.split(adsTest, 5)


In [9]:
import torch
torch.set_num_threads(1)
model, torch_utils = torch.hub.load(repo_or_dir='snakers4/silero-vad',
                              model='silero_vad',
                              force_reload=True)
(get_speech_timestamps,
 _, read_audio,
 *_) = torch_utils

Downloading: "https://github.com/snakers4/silero-vad/zipball/master" to /Users/david/.cache/torch/hub/master.zip


In [12]:
from feature_extraction.fe_configs import NormFactor
from spafe.features.mfcc import *

def get_log_features(sign, norm: NormFactor = None):
        features, _ = mel_spectrogram(sig=sign, fs=config.SAMPLE_RATE)
        features = zero_handling(features)
        features = np.log(features)
        if norm:
            features = (features - norm.means) / norm.std
            features = features - np.mean(features)
        return features

In [14]:
import dnnhmm
from audio_datastore.audio_datastore import subset
import processing.processing
import processing.process_method_base as process_method_base
import spafe.features.mfcc as mfcc
# from librosa.feature.spectral import mfcc

process_method = process_method_base.ProcessMethodBase(snr_db=40, reverb=False, model=model, get_speech_timestamps=get_speech_timestamps)

speakers = np.unique(adsEnroll.labels)
# normalisation
all_features = []
for i in range(len(speakers)):
    ads_train_subset = subset(adsEnroll, speakers[i])
    speaker_features = []
    for file in ads_train_subset.files:
        signal = process_method.pre_process(file)
        speaker_features.append(get_log_features(signal))
    speaker_features_flattened = np.array([item for sublist in speaker_features for item in sublist])
    all_features.append(speaker_features_flattened)

means = []
std = []
for feature in all_features:
    means.append(np.mean(feature, axis=0))
    std.append(np.std(feature, axis=0))

means = np.array(means)
means = np.mean(means, axis=0)

std = np.array(std)
std = np.mean(std, axis=0)

norm_factor = NormFactor(means, std)

In [20]:
from hmmlearn import hmm

speakers = np.unique(adsEnroll.labels)
speakers = speakers
hmms = {}
for i in range(len(speakers)):
    ads_train_subset = subset(adsEnroll, speakers[i])
    speaker_features = []
    for file in ads_train_subset.files:
        signal = process_method.pre_process(file)
        speaker_features.append(get_log_features(signal, norm_factor))
    speaker_features_flattened = np.array([item for sublist in speaker_features for item in sublist])
    new_hmm = hmm.GMMHMM(n_components=4, n_mix=2)
    new_hmm.fit(speaker_features_flattened)
    hmms[i] = new_hmm

In [24]:
# quick test
hmm_outer_scores = {}
for i in range(len(speakers)):
    ads_train_subset = subset(adsTest, speakers[i])
    speaker_features = []
    for file in ads_train_subset.files:
        signal = process_method.pre_process(file)
        speaker_features.append(get_log_features(signal, norm_factor))
    speaker_features_flattened = np.array([item for sublist in speaker_features for item in sublist])
    hmm_inner_scores = {}
    for y in range(len(hmms)):
        hmm_score = hmms[y].score(speaker_features_flattened)
        hmm_inner_scores[y] = hmm_score
    hmm_outer_scores[i] = hmm_inner_scores

# working so far !

In [ ]:
# now add dnn

